### Part 3: City district object

The city district object of pycity is an inheritance of the uesgraph object of uesgraphs package, which itself is an inheritance of networkx.Graph.

If you are not familiar with networkx.Graph or uesgraph, please go through the uesgraphs tutorial first. 
You can access it within the uesgraphs package, which is available under:
[UESGraphs](https://github.com/RWTH-EBC/uesgraphs)

You can install it locally via pip. Open a console and type:
'pip install -e your_path_to_uesgraphs' (with your local path to uesgraphs package)

If you have not installed it, yes, please do so now, because you are going to need it to execute the following lines.

In [31]:
#  Setting up environment

import pycity.classes.Timer as Timer
import pycity.classes.Weather as Weather
import pycity.classes.Prices as Prices
import pycity.classes.Environment as Env

#  Timestep in seconds
timestep = 3600

#  Total number of timesteps
timesteps_total = 365 * 24 * 3600 / timestep

#   Generate timer object for environment
timer = Timer.Timer(timeDiscretization=timestep, timestepsTotal=timesteps_total, 
                    timestepsUsedHorizon=timesteps_total, timestepsHorizon=timesteps_total)
#  Timer object holds timestep, number of timesteps as well as
#  forecast horizon

#  Generate weather object
weather = Weather.Weather(timer)
#  Weather object holds weather data, such as outdoor temperatures,
#  direct and diffuse radiation
#  Default TRY value is TRY2010_05_Jahr.dat
#  (Test reference year 2010 for region 5 in Germany)

#  Generate price object
price = Prices.Prices()
#  Holding energy prices and subsidies

#  Generate environment object
environment = Env.Environment(timer=timer, weather=weather, prices=price)

print(environment)

In [32]:
import pycity.classes.CityDistrict as CityDistrict

#  Generate city district object
city_district = CityDistrict.CityDistrict(environment)
#  You can generate a CityDistrict object without handing over the environment. This is done to enable some 
#  networkx functions for the citydistrict object. However, you should add the environment either way,
#  because you will not be able to work with your district later on, if the environment pointer is missing!
#  Thus, we recommmend to add the environment pointer directly!

print(city_district)

<uesgraphs.district object>


Now we will generate 3 single family buildings with loads and add them to the city district object

In [33]:
import random
import sympy.geometry.point as point

import pycity.classes.demand.DomesticHotWater as DomesticHotWater
import pycity.classes.demand.ElectricalDemand as ElectricalDemand
import pycity.classes.demand.SpaceHeating as SpaceHeating

import pycity.classes.demand.Apartment as Apartment
import pycity.classes.Building as Building


#  Loop to generate building objects
for i in range(3):
    
    #  Generate space heating object
    heat_demand = SpaceHeating.SpaceHeating(environment,
                                            method=1,  # Standard load profile
                                            livingArea=150,
                                            specificDemand=100)

    #  Generate electrical demand object
    el_demand = ElectricalDemand.ElectricalDemand(environment,
                                                  method=1,  # Standard load profile
                                                  annualDemand=3000)

    #  Generate domestic hot water demand object
    dhw_annex42 = DomesticHotWater.DomesticHotWater(environment,
                                                    tFlow=60,
                                                    thermal=True,
                                                    method=1,  # Annex 42
                                                    dailyConsumption=70,
                                                    supplyTemperature=25)
    
    #  Generate apartment and add load objects
    apartment = Apartment.Apartment(environment)
    apartment.addEntity(heat_demand)
    apartment.addMultipleEntities([el_demand, dhw_annex42])
    
    #  Generate building and add apartment
    building = Building.Building(environment)
    building.addEntity(entity=apartment)
    
    #  Generate sympy point positions (with random coordinates)
    position = point.Point(random.randint(0, 100), random.randint(0, 100))
    
    #  Add buildings to city district
    city_district.addEntity(entity=building, position=position)

print('Number of building entities:')
print(city_district.get_nb_of_building_entities())

Number of building entities:
3


You can also add pv- or wind farms to the city district:

In [34]:
import pycity.classes.supply.PV as PV

#  Generate PV field within city district
pv = PV.PV(environment, 20, 0.15)

#  Generate sympy point positions
position_1 = point.Point(0, 0)

#  Add PV fields to city district
city_district.addEntity(entity=pv, position=position_1)

print('Number of PV farms:')
print(city_district.get_nb_of_entities(entity_name='pv'))

Number of PV farms:
1


CAUTION: If you add pv- or windfarms to the city district, the number of buildings is not the same as tne length of nodelist_building. The reason is, that pv- and windfarms are also handled as building nodes within uesgraph. If you deal with building nodes, you should also check the _kind of entity within the node!

In [35]:
assert len(city_district.nodelist_building) == city_district.get_nb_of_building_entities(), ('Be aware, ' + 
                                                'that pv- and windfarms are also handled as building nodes in uesgraph! ')

AssertionError: Be aware, that pv- and windfarms are also handled as building nodes in uesgraph! 

In [36]:
#  Get complete node information of city district (possible, because city district is a graph)

print('Node information:')
print(city_district.nodes(data=True))
print()

print('Node numbers only:')
print(city_district.nodes())

Node information:
[(1001, {'entity': <pycity.classes.Building.Building object at 0x000000000C16A6D8>, 'name': 1001, 'is_supply_electricity': False, 'is_supply_gas': False, 'is_supply_heating': False, 'position': Point(55, 10), 'node_type': 'building', 'is_supply_cooling': False, 'is_supply_other': False}), (1002, {'entity': <pycity.classes.Building.Building object at 0x000000000C16A828>, 'name': 1002, 'is_supply_electricity': False, 'is_supply_gas': False, 'is_supply_heating': False, 'position': Point(26, 52), 'node_type': 'building', 'is_supply_cooling': False, 'is_supply_other': False}), (1003, {'entity': <pycity.classes.Building.Building object at 0x000000000C16A860>, 'name': 1003, 'is_supply_electricity': False, 'is_supply_gas': False, 'is_supply_heating': False, 'position': Point(8, 95), 'node_type': 'building', 'is_supply_cooling': False, 'is_supply_other': False}), (1004, {'entity': <pycity.classes.supply.PV.PV object at 0x000000000BBAB358>, 'name': 1004, 'is_supply_electricity'

CityDistrict holds further methods. Some are shown below:

In [37]:
print('Get number of building objects:')
print(city_district.get_nb_of_building_entities())
print()

print('Get list of nodes with building objects:')
print(city_district.get_list_build_entity_node_ids())

Get number of building objects:
3

Get list of nodes with building objects:
[1001, 1002, 1003]


It is also possible to extract aggregated load curves of all building objects

In [39]:
aggr_load_curve = city_district.get_aggr_space_h_power_curve()

print(aggr_load_curve)

ValueError: operands could not be broadcast together with shapes (876,) (8760,) (876,) 